In [11]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections
import re
from time import time
import math
import feather
from collections import defaultdict
import geopandas
from datetime import datetime
from sklearn.model_selection import train_test_split

#Using keras
import keras
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout

# others
from tqdm import tqdm

# original
from scripts.utils import load_rnn_data

# load data

In [12]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 1
predict_ts = 1

In [13]:
# load data as x and y of RNN
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path, window, predict_ts)

100%|██████████| 195/195 [00:10<00:00, 18.78it/s]


In [14]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))
print(len(y_datetime))

(1, 8, 195, 18611)
(1, 195, 18611)
195
18611


In [15]:
# np.save("x_all.npy", x_all)
# np.save("y_all.npy", y_all)

In [ ]:
# df = feather.read_dataframe("./features/features_binary_tract_2H.feather")

In [ ]:
# df.head()

In [ ]:
# df = df.sort_values(['geoid10_tract', 'datetime'])

In [ ]:
# df_x = df.drop(["datetime"],axis=1).reset_index(drop=True)
# df_x.head()

In [ ]:
# df_t = df.drop(["datetime","prcp","year","month","woy","dow","weekend","hour"],axis=1).reset_index(drop=True)
# df_t.head()

In [ ]:
# def _load_data(data, n_prev = 24):  

#     docX, docY = [], []
#     for i in range(len(data)-n_prev):
#         docX.append(data.iloc[i:i+n_prev].as_matrix())
#         docY.append(data.iloc[i+n_prev].as_matrix())
#     alsX = np.array(docX)
#     alsY = np.array(docY)

#     return alsX, alsY

# def train_test_split(df, test_size=0.1, n_prev = 24):  
#     """
#     This just splits data to training and testing parts
#     """
#     ntrn = round(len(df) * (1 - test_size))
#     ntrn = int(ntrn)
#     X_train, y_train = _load_data(df.iloc[0:ntrn], n_prev)
#     X_test, y_test = _load_data(df.iloc[ntrn:], n_prev)

#     return (X_train, y_train), (X_test, y_test)

In [ ]:
# x_data = np.array(df_x)
# t_data = np.array(df_t)

In [ ]:
# x_data.shape
# t_data.shape

In [ ]:
# maxlen = 24              # number of window (by hour)
# n_in = x_data.shape[1]   # the number of input（＝train data）
# n_out = t_data.shape[1]  # the number of output（=label data）
# len_seq = x_data.shape[0] - maxlen + 1
# data = []
# target = []
# for i in range(0, len_seq):
#     data.append(x_data[i:i+maxlen, :])
#     target.append(t_data[i+maxlen-1, :])

# x = np.array(data).reshape(len(data), maxlen, n_in)
# t = np.array(target).reshape(len(data), n_out)

print(x.shape, t.shape)

n_train = int(len(data)*0.9)              # length of train data
x_train,x_test = np.vsplit(x, [n_train])  # split into train and test data
t_train,t_test = np.vsplit(t, [n_train])  # split into train and test data

print(x_train.shape)
print(x_test.shape)
print(t_train.shape)
print(t_test.shape)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(t_train.shape)
print(t_test.shape)

In [ ]:

# (X_train, y_train), (X_test, y_test) = train_test_split(df, n_prev =length_of_sequences) 

In [ ]:
#Dense=4,when output is 4 types crimes. 
in_out_neurons = 11
length_of_sequences = 24
hidden_neurons = 50
dropout_rate = 0.2

model = Sequential()
model.add(LSTM(units=hidden_neurons, input_shape=(window, x_all.shape[1], x_all.shape[2]),
#                return_sequences=False
              ))
model.add(Dropout(dropout_rate))
model.add(Dense(units=195, activation="sigmoid"))

#using softmax
# model.add(Activation("sigmoid")) 
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

In [ ]:
model.summary()

In [ ]:
rnn = model.fit(x_all, y_all, epochs=1, batch_size=256)

In [ ]:
predicted = model.predict(x_test) 
df_predicted =  pd.DataFrame(predicted[:200])
data_predicted.head()

In [ ]:
data_actual = pd.DataFrame(y_test[:200])